## Step 1 - Imports
- Import bq, pandas and numpy

In [26]:
import google.datalab.bigquery as bq
import pandas as pd
import numpy as np

## Step 2 - Load Data


### Step 2.1 - Get the matches

In [27]:
%%bq query -n matches
SELECT match,home, away, HomeCode, AwayCode, HomeWin, Draw, AwayWin FROM `project-paul-the-octopus.kanjih.matches` LIMIT 1000

In [28]:
%%bq sample --count 5 --query matches

match,home,away,HomeCode,AwayCode,HomeWin,Draw,AwayWin
1,Egypt,Uruguay,EGY,URU,17,27,56
3,Saudi Arabia,Egypt,KSA,EGY,17,26,57
2,Russia,Egypt,RUS,EGY,49,28,23
1,Russia,Saudi Arabia,RUS,KSA,72,20,8
2,Uruguay,Saudi Arabia,URU,KSA,77,17,6


### Step 2.2 - Put the data in a dataframe



In [32]:
df = matches.execute(output_options=bq.QueryOutput.dataframe()).result()
len(df) 

48

## Step 3 - Prediction

### Step 3.1 - Predication Layer 1 (Generate winner by score match)

In [33]:
df['homeScore'] = np.where((df["HomeWin"]-df["AwayWin"]) > 10, 1, 0)
df['awayScore'] = np.where((df["AwayWin"]-df["HomeWin"]) > 10, 1, 0)
df

,match,home,away,HomeCode,AwayCode,HomeWin,Draw,AwayWin,homeScore,awayScore
0,1,Tunisia,England,TUN,ENG,10,19,71,0,1
1,1,Egypt,Uruguay,EGY,URU,17,27,56,0,1
2,1,Portugal,Spain,POR,ESP,22,27,51,0,1
3,1,Peru,Denmark,PER,DEN,25,28,47,0,1
4,1,Costa Rica,Serbia,CRC,SRB,25,28,47,0,1
5,1,Morocco,Iran,MAR,IRN,42,31,27,1,0
6,1,Poland,Senegal,POL,SEN,42,30,28,1,0
7,1,Sweden,South Korea,SWE,KOR,45,29,26,1,0
8,1,Croatia,Nigeria,CRO,NGA,51,28,21,1,0
9,1,Colombia,Japan,COL,JPN,55,27,18,1,0


## STEP 4 - Clean Data

In [36]:
df_to_print = df[['home', 'homeScore', 'awayScore','away']].copy()
df_to_print

,home,homeScore,awayScore,away
0,Tunisia,0,1,England
1,Egypt,0,1,Uruguay
2,Portugal,0,1,Spain
3,Peru,0,1,Denmark
4,Costa Rica,0,1,Serbia
5,Morocco,1,0,Iran
6,Poland,1,0,Senegal
7,Sweden,1,0,South Korea
8,Croatia,1,0,Nigeria
9,Colombia,1,0,Japan


## Step 5 - Print CSV

In [37]:
%%gcs write --variable df_to_print --object 'gs://paul_kanjih_v1/result_v1.csv'